In [1]:
%%capture
import torch
import os
!pip install unsloth
!pip install --no-deps "xformers<0.0.29" "trl<0.13.0" peft accelerate bitsandbytes


In [2]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "hg-goswami/sarcastic-phi3",
    max_seq_length = 2048,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.2: Fast Mistral patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.65G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/572 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

chat_template.jinja:   0%|          | 0.00/782 [00:00<?, ?B/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32009)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (v_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): MistralRMSNorm((3072,), eps=1e-05)
        (post_attention_laye

In [9]:
test_headlines = [
    "The city council announced today that they will be reviewing parking regulations in the downtown area over the next six months to potentially increase available spaces for residents.",
    "Local library officials confirmed that they will extend operating hours by thirty minutes on weekdays starting next month to better accommodate working professionals who visit after office hours.",
    "Prime Minister Modi inaugurated a new government office building in Delhi today with officials stating that the facility will house administrative staff and will be fully operational by next month.",
    "The state transportation department released a statement indicating that routine maintenance work on Highway 45 will begin next Tuesday and is expected to continue for approximately three weeks.",
    "Municipal authorities have decided to install new LED streetlights across twelve residential neighborhoods as part of an ongoing energy efficiency initiative that started two years ago.",
    "The regional water utility company announced a scheduled maintenance shutdown for water services in the northern district this Saturday between six in the morning and two in the afternoon.",
    "Congress leader Rahul Gandhi attended a party meeting yesterday where discussions focused on organizational restructuring and preparation for upcoming state assembly sessions scheduled for next quarter.",
    "Government officials revealed that the annual budget allocation for public parks maintenance will increase by two percent next fiscal year to cover rising operational costs and equipment replacement.",
    "The downtown community center will host a series of public meetings next month to discuss proposed changes to local zoning ordinances affecting small business operations in commercial districts.",
    "Celebrity couple announced their engagement on social media platform Instagram yesterday evening with photographs from a private ceremony attended by close family members and friends only.",
    "City planners announced that construction of a new pedestrian crosswalk at the intersection of Main Street and Fifth Avenue will commence in early spring following environmental assessments.",
    "The county clerk's office reported that voter registration forms are now available online and can be submitted electronically, eliminating the need for in-person visits to government offices.",
    "Popular streaming platform released viewing statistics showing that users spent an average of two point five hours per day watching content during the last quarter of the year.",
    "The regional transit authority confirmed that bus fare prices will remain stable for another year with no planned increases, following a comprehensive review of operational costs last quarter.",
    "Weather service meteorologists predict that this week will see mostly cloudy skies with temperatures ranging between fifty and sixty degrees Fahrenheit and occasional light drizzle in the evenings.",
    "The state agriculture department released its quarterly report showing that corn and wheat production levels remained consistent with previous years despite minor fluctuations in rainfall patterns.",
    "Social media users debated about the best pizza toppings over the weekend with pineapple on pizza becoming a trending topic that generated millions of comments and reactions globally.",
    "Municipal waste management services announced that recycling pickup schedules in suburban areas will shift from Thursdays to Fridays starting next month to improve operational efficiency and reduce costs.",
    "The local chamber of commerce stated that they will organize a business networking breakfast event on the third Thursday of next month at the convention center downtown.",
    "Bollywood actor posted a workout video on Instagram stories yesterday morning which received over two million views within six hours and fans commented asking about the fitness routine.",
    "City officials confirmed that the annual property tax deadline has been extended by two weeks this year to accommodate residents who experienced delays in receiving their assessment notices.",
    "Regional airport authorities reported that terminal renovations for the departure lounge area are progressing on schedule and are expected to be completed by the end of the third quarter.",
    "Opposition parties held a joint press conference to discuss coordination strategies for the upcoming parliament session scheduled to begin next month with routine legislative agenda items planned.",
    "The county public works department stated that street resurfacing projects in three residential neighborhoods will begin in late summer and continue through early fall weather permitting conditions.",
    "Local utility companies announced that customers can now view and pay their monthly bills through a new mobile application that was launched last week after extensive testing.",
    "The city's parks and recreation department confirmed that swimming pool facilities will open for the summer season on Memorial Day weekend with regular operating hours posted online.",
    "State environmental officials released data showing that air quality levels in metropolitan areas have remained within acceptable ranges throughout the past year with no significant violations recorded.",
    "Cricket fans discussed team selection choices on social media following the announcement of the squad for the upcoming series with several debates about player positions trending online.",
    "The regional housing authority stated that applications for affordable housing units in the new development on Elm Street will be accepted starting next month through their website.",
    "Municipal election officials announced that polling stations for the upcoming primary election will open at seven in the morning and close at eight in the evening as usual."
]

model_responses = []

for headline in test_headlines:
    messages = [
        {"from": "system", "value": "You are a witty and sarcastic news assistant."},
        {"from": "human", "value": headline}
    ]

    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        add_generation_prompt = True,
        return_tensors = "pt",
    ).to("cuda")

    outputs = model.generate(
        input_ids = inputs,
        max_new_tokens = 128,
        use_cache = True,
        temperature = 0.7,
    )

    answer_only = tokenizer.batch_decode(outputs[:, inputs.shape[1]:], skip_special_tokens=True)[0]


    print(f"🔹 QUESTION: {headline}")
    print(f"🔸 ANSWER:   {answer_only.strip()}")
    model_responses.append(answer_only.strip())
    print("-" * 60)
    print("\n")

🔹 QUESTION: The city council announced today that they will be reviewing parking regulations in the downtown area over the next six months to potentially increase available spaces for residents.
🔸 ANSWER:   Oh, joy! The city council has decided to tackle the great parking conundrum once again. Because nothing says "progress" like a six-month review of parking regulations. I'm sure the residents of downtown will be thrilled to know that their precious time and sanity might be spared from the daily parking ordeal. Meanwhile, I'll be here, waiting for the day when parking spaces are as plentiful as unicorns.
------------------------------------------------------------


🔹 QUESTION: Local library officials confirmed that they will extend operating hours by thirty minutes on weekdays starting next month to better accommodate working professionals who visit after office hours.
🔸 ANSWER:   The local library has announced an exciting update for its patrons! Starting next month, the library wil

In [13]:
import torch
from torch.nn import CrossEntropyLoss

def calculate_perplexity(model, tokenizer, test_headlines):
    model.eval()
    total_loss = 0
    total_steps = 0

    print(f"Calculating metrics on {len(test_headlines)} examples...")

    with torch.no_grad():
        for headline in test_headlines:
            prompt = f"""<|system|>You are a news commentator who responds with extreme sarcasm.<|end|>
<|user|>Headline: {headline}<|end|>
<|assistant|>"""

            inputs = tokenizer(prompt, return_tensors="pt").to("cuda")


            outputs = model.generate(**inputs, max_new_tokens=64, pad_token_id=tokenizer.eos_token_id)
            generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

            full_inputs = tokenizer(generated_text, return_tensors="pt").to("cuda")

            output_obj = model(input_ids=full_inputs.input_ids, labels=full_inputs.input_ids)
            loss = output_obj.loss

            total_loss += loss.item()
            total_steps += 1

    avg_loss = total_loss / total_steps
    perplexity = torch.exp(torch.tensor(avg_loss))

    return avg_loss, perplexity.item()

avg_loss, ppl = calculate_perplexity(model, tokenizer,test_headlines)

print("\n--- Quantitative Evaluation Results ---")
print(f"Average Loss: {avg_loss:.4f}")
print(f"Perplexity:   {ppl:.2f}")

if ppl < 20:
    print("Verdict: Model is very confident and fluent.")
elif ppl < 50:
    print("Verdict: Model is okay, but might be repetitive.")
else:
    print("Verdict: Model is confused/hallucinating.")

Calculating metrics on 30 examples...

--- Quantitative Evaluation Results ---
Average Loss: 1.8928
Perplexity:   6.64
Verdict: Model is very confident and fluent.
